<br>


# 基于 PyTorch 构建卷积神经网络 CNN

<br>

## 0. 概述

<br>

<font color=black size=3 face=雅黑>　　在前面两项实验内容中，我们已经学习了 PyTorch 的基本数据类型 tensor 及其相关操作，并练习了如何通过 PyTorch 读入并处理数据集。下面，我们就正式开始学习如何基于 PyTorch 搭建一个神经网络，并在 MNIST 数据集上进行训练和测试。
    
<br>

<font color=black size=3 face=雅黑>　　总体来讲，我们在这部分实验中要进行的操作包括以下几点：

<font color=black size=3 face=雅黑>　　1) 　读取和处理数据集
    
<font color=black size=3 face=雅黑>　　2) 　定义一个包含可训练参数的神经网络
    
<font color=black size=3 face=雅黑>　　3) 　从数据集中取出一个批次 (batch) 的样本传递给神经网络

<font color=black size=3 face=雅黑>　　4) 　通过神经网络处理输入样本

<font color=black size=3 face=雅黑>　　5) 　计算损失 (loss)

<font color=black size=3 face=雅黑>　　6) 　反向传播计算偏导

<font color=black size=3 face=雅黑>　　7) 　更新网络的参数 (一个简单的更新方法：weight = weight - learning_rate *gradient)
    
<font color=black size=3 face=雅黑>　　8) 　重复 3) 到 7) 步，不断调整网络参数

<br>

## 1. 导入 PyTorch 及其他相关库

In [71]:
import torch
import torchvision
import numpy as np

<font color=black size=3 face=雅黑>查看 PyTorch 版本，是否可以使用 GPU，及 CUDA 的版本。

In [72]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

1.5.0
True
10.2


## 2. 准备数据集 (Data Preparation)

<br>

In [73]:
batch_size = 100  # 设置训练集和测试集的 batch size，即每批次将参与运算的样本数

train_set = torchvision.datasets.MNIST('./dataset_mnist', train=True, download=True,
                                       transform=torchvision.transforms.Compose([
                                           torchvision.transforms.ToTensor(),
                                           torchvision.transforms.Normalize(
                                               (0.1307,), (0.3081,)
                                           )
                                       ])
)

##################### please finish the code ########################

# 请把之前在“1_mnist_dataset_import.ipynb”中写的 test_set 代码复制在这里
# test_set = XXX
test_set = torchvision.datasets.MNIST('./dataset_mnist', train=False, download=True,
                                      transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize(
                                              (0.1307,), (0.3081,)
                                          )
                                      ])
                                      )

################################ end ################################

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

<br>

## 3.  定义一个神经网络

<br>

<font color=black size=3 face=雅黑>　　为了构建神经网络，我们将使用 torch.nn 包，它是 PyTorch 的神经网络库。我们通常会将 torch.nn 包直接导入并命名为 nn。

In [74]:
import torch.nn as nn

<br>
<font color=black size=3 face=雅黑>　　PyTorch 的神经网络库包含了构建神经网络所需的所有典型组件。我们构建一个神经网络使用的最主要组件是层，所以 PyTorch 的神经网络库包含了帮助我们构造层的类。在神经网络包里有一类被称为“模块” (nn.Module) 的特殊类，它是所有神经网络模块的母类。PyTorch 中所有的神经网络层均继承了 nn.Module。    

<br>    
    
<font color=black size=3 face=雅黑>　　神经网络中的每一层都包含两个主要组成部分，第一是该层参数的集合 (weights and biases)，第二是该层进行的变换 (transformation)。其中，进行的变换主要指前向传播过程，每一个 PyTorch nn.Module 都有一个 forward() 函数来定义其前向传播。因此，当我们实现一个自定义的神经网络 (nn.Module 子类) 时，也需要编写其 forward() 函数。值得注意的是，得益于 PyTorch 的自动求导功能，我们并不需要像“实验一”中一样自己编写反向传播的 backward() 函数，PyTorch 会自动帮我们完成。（关于这一点，我们会在之后的实验中详细为大家介绍。）
    
<br>

<font color=black size=3 face=雅黑>　　总结起来，在 PyTorch 中定义神经网络包括如下几个步骤：
    
<font color=black size=3 face=雅黑>　　　1) 基于 nn.Module 创建一个类 (extend the nn.Module base class)　
    
<font color=black size=3 face=雅黑>　　　2) 在类构造函数中，将网络的层定义为类属性 (define layers as class attributes)
    
<font color=black size=3 face=雅黑>　　　3) 定义网络的前向传输 (implement the forward() method)

<br>

<font color=black size=3 face=雅黑>　　下面，我们就可以创建一个类来表示一个简单的神经网络。

<br>

In [75]:
class Network0(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer = None
    
    def forward(self, t):
        t = self.layer(t)
        return t

<font color=black size=3 face=雅黑>　　在这个例子中，我们扩展了 nn.Module 基类。它在构造函数中有一个虚拟层 (self.layer)，并在 forward() 函数中实现了一个虚拟的前向传播过程。forward() 接收来自神经网络输入端的张量 t，对其进行张量变换 (transformation)，并将变换后的结果返回。

<br>

<font color=black size=3 face=雅黑>　　现在，让我们用一些真正的层来替代这个虚拟层，这些层将由PyTorch 神经网络库提供。我们将建立一个卷积神经网络，包含两个卷积层和三个全连接层。其中，卷积层我们将用 torch.nn 的内建层类 nn.Conv2d 来实现，全连接层将用 nn.Linear 来实现。

    
<font color=black size=3 face=雅黑>　　nn.Conv2d 可见 PyTorch 官网：   https://pytorch.org/docs/master/generated/torch.nn.Conv2d.html#torch.nn.Conv2d
    
<font color=black size=3 face=雅黑>　　nn.Linear 可见 PyTorch 官网：
https://pytorch.org/docs/master/generated/torch.nn.Linear.html#torch.nn.Linear
    
<br>  

<font color=black size=3 face=雅黑>　　假设我们将使用 MNIST 数据集，每张图片为一个 28\*28 的灰度图。在即将定义的神经网络中，每个卷积层之后我们都会进行池化操作 (pooling)，池化的 kernel 大小和 stride 均取 2（即每次将对 2*2 的元素进行 max pooling）。 池化层由于不包含可训练的参数，因此不作为 Network1 的类属性（class attribute），而是会在之后写入 forward() 函数里。
    
<br>
    
<font color=black size=3 face=雅黑>　　**请大家阅读以下代码，在实验报告中：**
    
    
<font color=black size=3 face=雅黑>　　  **1) 画出对应的网络结构（包含各层权重的形状）；**
    
    
<font color=black size=3 face=雅黑>　　  **2) 注明当一张图片输入进网络后各层输入、输出值的形状；（由于不考虑批量化处理图片，因此卷积层激活值以三维张量表示，全连接层激活值以一维张量表示即可。）**
    
<font color=black size=3 face=雅黑>　　  **3) 补全 “self.fc1”的代码。（[提示]：需要计算 fc1 层的输入激活向量的长度。）**
    
<br>

In [76]:
class Network1(nn.Module): 
    def __init__(self):
        super(Network1, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1,  # 输入通道数
                               out_channels=6,  # 输出通道数
                               kernel_size=5)  # kernel 大小 （对应四维权重张量的 dim2/dim3）
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        ##################### please finish the code ########################
        
        # self.fc1 = nn.Linear(in_features=XXX,  # 输入激活向量的长度
        #                     out_features=120)  # 输出激活向量的长度    
        self.fc1 = nn.Linear(in_features=192,  # 输入激活向量的长度
                             out_features=120)  # 输出激活向量的长度
        ################################ end ################################
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        # we will implement the forward path later
        t = self.pool1(F.relu(self.conv1(t)))
        t = self.pool2(F.relu(self.conv2(t)))
        t = self.fc1(t)
        t = self.fc2(t)
        t = self.out(t)
        return t

<font color=black size=3 face=雅黑>　　查看定义好的网络结构

In [77]:
network = Network1()
print(network)

Network1(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)


<font color=black size=3 face=雅黑>　　查看卷积层及其参数

In [78]:
print(network.conv1)
print(network.conv2)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))


In [79]:
print(network.conv1.weight)  # check the weights of conv1

Parameter containing:
tensor([[[[-0.1374, -0.1571,  0.1716, -0.1089, -0.0602],
          [ 0.1650, -0.0900, -0.1596,  0.0584, -0.1780],
          [ 0.1025,  0.0006, -0.0053,  0.1643,  0.0172],
          [-0.0898,  0.1827, -0.0343, -0.0658,  0.1190],
          [-0.0979, -0.0075,  0.1803, -0.1945, -0.0229]]],


        [[[ 0.0404,  0.0069, -0.1702,  0.1188, -0.0949],
          [ 0.0229,  0.0759, -0.1565,  0.0287,  0.0472],
          [ 0.0144, -0.1261, -0.1770, -0.1682,  0.1396],
          [ 0.1011, -0.0097,  0.0503, -0.0841,  0.1170],
          [-0.1369,  0.0372, -0.1275, -0.1588, -0.1501]]],


        [[[ 0.1558, -0.1111, -0.0622,  0.1836, -0.0074],
          [ 0.0988, -0.0321, -0.1011,  0.0416,  0.0888],
          [-0.0324,  0.1643,  0.0623,  0.1197, -0.1037],
          [-0.1130,  0.0424, -0.1538,  0.1787,  0.0648],
          [ 0.1192, -0.0831,  0.0249,  0.1153,  0.0868]]],


        [[[ 0.1547, -0.0096,  0.0841, -0.1166,  0.1698],
          [-0.1528,  0.0019, -0.1681, -0.0795,  0.0812

In [80]:
print(network.conv1.weight.shape)  # check the shape of conv1's weight tensor
print(network.conv1.weight[0])  # check weigth[0]
print(network.conv1.weight[0].shape)  # check the shape of weight[0]

torch.Size([6, 1, 5, 5])
tensor([[[-0.1374, -0.1571,  0.1716, -0.1089, -0.0602],
         [ 0.1650, -0.0900, -0.1596,  0.0584, -0.1780],
         [ 0.1025,  0.0006, -0.0053,  0.1643,  0.0172],
         [-0.0898,  0.1827, -0.0343, -0.0658,  0.1190],
         [-0.0979, -0.0075,  0.1803, -0.1945, -0.0229]]],
       grad_fn=<SelectBackward>)
torch.Size([1, 5, 5])


<font color=black size=3 face=雅黑>　　查看全连接层及其参数

In [81]:
# weight tensor for fc layers
print(network.fc1.weight.shape)
print(network.fc1.weight[0].shape)

torch.Size([120, 192])
torch.Size([192])


<font color=black size=3 face=雅黑>　　查看整个网络的参数

In [82]:
# 方式一
for name, param in network.named_parameters():
    print(name, '\t\t', param.shape)
print("")    

#方式二
for name in network.state_dict():  # state dictionary
    print(name, '\t\t', network.state_dict()[name].shape)

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


In [83]:
# 查看 conv1 的参数
print(network.state_dict()["conv1.weight"].shape)
print(network.state_dict()["conv1.weight"])

torch.Size([6, 1, 5, 5])
tensor([[[[-0.1374, -0.1571,  0.1716, -0.1089, -0.0602],
          [ 0.1650, -0.0900, -0.1596,  0.0584, -0.1780],
          [ 0.1025,  0.0006, -0.0053,  0.1643,  0.0172],
          [-0.0898,  0.1827, -0.0343, -0.0658,  0.1190],
          [-0.0979, -0.0075,  0.1803, -0.1945, -0.0229]]],


        [[[ 0.0404,  0.0069, -0.1702,  0.1188, -0.0949],
          [ 0.0229,  0.0759, -0.1565,  0.0287,  0.0472],
          [ 0.0144, -0.1261, -0.1770, -0.1682,  0.1396],
          [ 0.1011, -0.0097,  0.0503, -0.0841,  0.1170],
          [-0.1369,  0.0372, -0.1275, -0.1588, -0.1501]]],


        [[[ 0.1558, -0.1111, -0.0622,  0.1836, -0.0074],
          [ 0.0988, -0.0321, -0.1011,  0.0416,  0.0888],
          [-0.0324,  0.1643,  0.0623,  0.1197, -0.1037],
          [-0.1130,  0.0424, -0.1538,  0.1787,  0.0648],
          [ 0.1192, -0.0831,  0.0249,  0.1153,  0.0868]]],


        [[[ 0.1547, -0.0096,  0.0841, -0.1166,  0.1698],
          [-0.1528,  0.0019, -0.1681, -0.0795,  0.0

<br>

<font color=black size=3 face=雅黑>　　网络的卷积层和全连接层定义好了。现在，让我们来实现 forward() 函数。
    
<font color=black size=3 face=雅黑>　　神经网络 forward() 函数的编写通常需要使用来自 nn.functional 的函数，这个包为我们提供了许多实现前向传播时需要的神经网络操作，例如激活函数 ReLU 和池化操作 Max Pooling。值得注意的是，虽然我们常称它们为“激活层”和“池化层”，但实际上它们都是在执行某种操作，并不包含可训练的参数。因此，它们在 PyTorch 中并没有被作为层来处理，也没有继承 nn.Module 基类，而是包含在 nn.functional 中。
    
<br>

<font color=black size=3 face=雅黑>　　**请在实验报告中展示补全后的 Network2.forward()，并说明这样写的原因。**

In [84]:
import torch.nn.functional as F

In [85]:
class Network2(nn.Module):
    def __init__(self):
        super(Network2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        ##################### please finish the code ########################
        
        # 请把之前 Network1 的 self.fc1 复制到这里：
        # self.fc1 = nn.Linear(in_features=XXX, out_features=120)
        self.fc1 = nn.Linear(in_features=192,  # 输入激活向量的长度
                             out_features=120)  # 输出激活向量的长度   
        ################################ end ################################
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        
        # conv1
        t = self.conv1(t)
        t = F.relu(t)  # 这里我们用到了 nn.functional 包中的激活函数 relu
        t = F.max_pool2d(t, kernel_size=2, stride=2)  # 这里我们用到了 nn.functional 包中的池化函数 max_pool2d （max pooling）
        
        # conv2
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # 特别注意：最后一个卷积层的输出在流入全连接层之前，要对激活值的维度进行变换，变换后的张量
        # 是二维张量，其中 dim0 对应该批次中的不同样本，dim1 对应每个样本流向全连接层的输入激活向量。
        # 请完成如下代码。
        
        ##################### please finish the code ########################

        # dim0：一个批次的样本数
        # dim1：每一个样本的输入激活向量长度
        
        # t = t.reshape(XXX,XXX)
        t = t.reshape((t.shape[0], -1))
        ################################ end ################################

        # fc1
        t = self.fc1(t)
        t = F.relu(t)
        
        # fc2
        t = self.fc2(t)
        t = F.relu(t)
        
        # output layer
        t = self.out(t)
        
        return t

In [86]:
class Network2(nn.Module):
    def __init__(self):
        super(Network2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
        self.fc1 = nn.Linear(in_features=192,  # 输入激活向量的长度  # 由模型网络计算得出的 192
                             out_features=120)  # 输出激活向量的长度   

        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        # conv1
        t = self.conv1(t)
        t = F.relu(t)  # 这里我们用到了 nn.functional 包中的激活函数 relu
        t = F.max_pool2d(t, kernel_size=2, stride=2)  # 这里我们用到了 nn.functional 包中的池化函数 max_pool2d （max pooling）

        # conv2
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.reshape((t.shape[0], -1))  # 此处将函数打平。dim0 对应该批次中的不同样本，dim1 对应每个样本流向全连接层的输入激活向量。
        # fc1
        t = self.fc1(t)
        t = F.relu(t)
        # fc2
        t = self.fc2(t)
        t = F.relu(t)
        # output layer
        t = self.out(t)
        
        return t

<br>

<font color=black size=3 face=雅黑>　　在 PyTorch 中，当我们直接调用对象名称 conv1/conv2/fc1/fc2 并将输入 t 传递给它时，对应的 forward() 函数将被调用。因此，在上述代码中， 我们直接将输入 t 传递给了 conv1，而没有使用 t = self.conv1.forward(t)。
    
<br>
    
<font color=black size=3 face=雅黑>　　到这里，我们已经完成了神经网络的构建。下一步，我们将训练这个神经网络来处理手写字体识别任务。
    

<font color=black size=3 face=雅黑>　　在开始训练之前，我们先来看一下这个神经网络的正向传播过程。


In [119]:
# 关掉 PyTorch 的自动求导功能
torch.set_grad_enabled(False)
# torch.set_grad_enabled(True)

In [125]:
network = Network2()

# 我们取一批图片给 network
batch = next(iter(train_loader))  # DataLoader object: train_loader
images, labels = batch
print(images.shape)  # rank-4 tensor
print(labels.shape)

output = network(images)

torch.Size([100, 1, 28, 28])
torch.Size([100])


In [126]:
# 查看 output 形状: rank-2 tensor （[batch_size, # of classes]）
output.shape

torch.Size([100, 10])

In [127]:
# F.softmax(t, dim=1）会为每个预测类返回一个概率值，这些概率值之和等于1。我们没有将这一步放在 Network2.forward()
# 中来做，是因为我们在训练网络将使用交叉熵损失函数 nn.CrossEntropyLoss，它会在其输入上隐式的执行一个 Softmax 操作。
output_prob = F.softmax(output, dim=1)

# 在之前的练习中我们已经知道，torch.max(t, dim=1) 函数会返回两个 tensor，第一个 tensor 是每行的最大值；第二个
# tensor是每行最大值的索引，反映了批处理中每个图像的预测标签。
scores, predict_class = torch.max(output_prob, dim=1)

print("scores: \n", scores)
print("predicted labels: \n", predict_class)

scores: 
 tensor([0.1084, 0.1078, 0.1088, 0.1085, 0.1088, 0.1095, 0.1084, 0.1076, 0.1094,
        0.1076, 0.1070, 0.1107, 0.1080, 0.1087, 0.1077, 0.1092, 0.1087, 0.1095,
        0.1072, 0.1066, 0.1083, 0.1072, 0.1072, 0.1074, 0.1081, 0.1087, 0.1069,
        0.1088, 0.1071, 0.1122, 0.1062, 0.1099, 0.1079, 0.1072, 0.1085, 0.1081,
        0.1078, 0.1102, 0.1080, 0.1076, 0.1089, 0.1089, 0.1081, 0.1102, 0.1086,
        0.1085, 0.1087, 0.1089, 0.1077, 0.1102, 0.1093, 0.1079, 0.1087, 0.1075,
        0.1113, 0.1072, 0.1085, 0.1066, 0.1074, 0.1085, 0.1088, 0.1116, 0.1086,
        0.1087, 0.1093, 0.1089, 0.1078, 0.1086, 0.1110, 0.1100, 0.1089, 0.1056,
        0.1087, 0.1076, 0.1090, 0.1071, 0.1073, 0.1082, 0.1088, 0.1095, 0.1091,
        0.1082, 0.1096, 0.1085, 0.1064, 0.1094, 0.1094, 0.1081, 0.1092, 0.1102,
        0.1075, 0.1082, 0.1101, 0.1058, 0.1094, 0.1073, 0.1099, 0.1115, 0.1104,
        0.1092], grad_fn=<MaxBackward0>)
predicted labels: 
 tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,

In [128]:
# 查看实际标签
print("actual labels: \n", labels)

actual labels: 
 tensor([8, 1, 8, 4, 1, 6, 9, 2, 2, 3, 7, 0, 8, 1, 2, 6, 6, 5, 6, 7, 1, 9, 4, 8,
        2, 7, 2, 9, 2, 2, 7, 1, 4, 2, 0, 4, 2, 5, 2, 2, 9, 5, 4, 1, 9, 4, 1, 4,
        2, 6, 9, 3, 8, 9, 6, 4, 1, 7, 0, 6, 6, 0, 6, 1, 0, 2, 3, 1, 6, 0, 1, 9,
        1, 1, 5, 9, 4, 9, 4, 7, 2, 3, 5, 8, 5, 0, 6, 9, 1, 8, 7, 6, 2, 3, 9, 2,
        3, 5, 5, 8])


In [129]:
# 计算这一批样本识别的“准确率”
correct_predictions = (predict_class == labels).sum().item()  # 识别正确的次数
print("correct predictions: ", correct_predictions)
print("\"accuracy\": ", correct_predictions/len(labels)*100, "%") 

correct predictions:  8
"accuracy":  8.0 %


<font color=black size=3 face=雅黑>　　**把这一部分代码（从"network = Network2()"开始）多跑几次你会发现，每次 "scores"、"predict_class" 等都会发生变化，请思考这是为什么？另外，尽管"scores"每一次都不同，但其大致取值却都差不多，这又是为什么？请在实验报告中说明。**

<br>

In [64]:
# 现在我们可以将自动求导功能打开，为训练做好准备
torch.set_grad_enabled(True)

<br>

## 4.  训练神经网络

<br>

<font color=black size=3 face=雅黑>　　在这一部分中，我们将在数据集 MNIST 上训练构建好的神经网络。
    
<br>

<font color=black size=3 face=雅黑>　　在定义完神经网络之后，我们还需要在 torch.nn 包中选择合适的损失函数 (loss function) 来评估网络输出与理想值之间的差别。常用的损失函数都已经定义在了 torch.nn 中，比如均方误差 (nn.MSELoss)、多分类的交叉熵 (nn.CrossEntropyLoss) 及二分类的交叉熵 (nn.BCELoss)，等等。由于我们将进行手写字体0到9的识别，本次实验我们将选用交叉熵损失函数。
    
<font color=black size=3 face=雅黑>　　一个损失函数需要一对输入：模型的实际输出和目标输出。基于此，损失函数将计算一个值 (loss) 来评估实际输出距离目标输出有多远。

In [65]:
loss_func = nn.CrossEntropyLoss()

<font color=black size=3 face=雅黑>　　除了损失函数以外，我们还需要一个能更直观的反映分类准确性的函数，方便我们了解训练的情况。请大家阅读并理解以下 get_num_correct 的代码每一步在做什么。

In [66]:
def get_num_correct(preds, labels):  # get the number of correct times
    return preds.argmax(dim=1).eq(labels).sum().item()

<font color=black size=3 face=雅黑>　　在深度学习的训练过程中，我们需要通过不断调整参数使得损失最小化。优化算法就是一种调整模型参数更新的策略。在这里，我们需要用到一个新的包：torch.optim，其中“optim”是“optimizer”的缩写，它包含了多种常用的优化算法。


In [67]:
import torch.optim as optim

<font color=black size=3 face=雅黑>　　本次实验我们将使用随机梯度下降法 (SGD) 来对网络进行优化。

In [68]:
optimizer = optim.SGD(network.parameters(), lr=0.1)  # lr: 学习率

<font color=black size=3 face=雅黑>　　现在我们已经做好了训练的前期准备。在开始大规模的训练前，我们先来观察一个批次 (batch) 的图片是如何被训练的。

<font color=black size=3 face=雅黑>　　首先，我们取出一个 batch 的样本，传递给 network，并计算损失。

In [69]:
batch = next(iter(train_loader))
images, labels = batch

preds = network(images)
loss = loss_func(preds, labels)
loss.item()

2.3154964447021484

<font color=black size=3 face=雅黑>　　现在我们有了 loss，下一步可以计算偏导 (gradients)。为了计算偏导，我们会在损失张量上调用反向函数 loss.backward()，PyTorch 会自动帮我们做相关的计算。

In [70]:
loss.backward()

<font color=black size=3 face=雅黑>　　我们可以看看100张图片中分类正确的次数：

In [71]:
get_num_correct(preds, labels) 

10

<font color=black size=3 face=雅黑>　　下一步是使用这些偏导来更新网络的权重。

In [72]:
optimizer.step()  # 更新参数的方法

<font color=black size=3 face=雅黑>　　我们将同一批样本再次传递给更新后的神经网络，看看 loss 和分类正确的次数有什么变化。

In [73]:
preds = network(images)
loss = loss_func(preds, labels)
print("loss: ", loss.item())
print("number of correct times: ", get_num_correct(preds, labels))

loss:  2.3077101707458496
number of correct times:  10


<font color=black size=3 face=雅黑>　　可以看到 loss 有些微的下降，说明我们确实是在朝着损失减小的方向调整参数。下面我们来完整实现整个训练过程。
    
<font color=black size=3 face=雅黑>　　**请大家补全以下代码，在实验报告中展示相关代码并做出说明，训练结果请截图保存。**    
    
<br>

In [74]:
total_epochs = 10

for epoch in range(total_epochs):  # 训练周期

    total_loss = 0
    total_train_correct = 0

    for batch in train_loader:  # get a batch from the dataloader
        
        
        # 读取样本数据，完成正向传播，计算损失
        
        ##################### please finish the code ########################
        
        # images, labels = XXX        
        # preds = XXX
        # loss = XXX
        images, labels = batch
        preds = network(images)
        loss = loss_func(preds, labels)
        ################################ end ################################

                
        # 下面这行非常重要，它使得优化器 (optimizer) 将权重的偏导重新归零；
        # 如果不归零，那么在反向传播时，计算出来的偏导会累加在原先的偏导上，
        # 造成错误。
        optimizer.zero_grad()
        
        
        # 完成反向传播，更新参数
        
        ##################### please finish the code ########################
        
        # 反向传播 （一行代码）
        loss.backward()
        
        # 更新参数 （一行代码）        
        optimizer.step()

        ################################ end ################################
  
        total_loss += loss.item()
        total_train_correct += get_num_correct(preds, labels)
    
    print("epoch:", epoch, 
          "correct times:", total_train_correct,
          f"training accuracy:", "%.3f" %(total_train_correct/len(train_set)*100), "%", 
          "total_loss:", "%.3f" %total_loss)

epoch: 0 correct times: 53153 training accuracy: 88.588 % total_loss: 217.655
epoch: 1 correct times: 58485 training accuracy: 97.475 % total_loss: 48.118
epoch: 2 correct times: 58939 training accuracy: 98.232 % total_loss: 33.544
epoch: 3 correct times: 59203 training accuracy: 98.672 % total_loss: 26.443
epoch: 4 correct times: 59294 training accuracy: 98.823 % total_loss: 22.080
epoch: 5 correct times: 59404 training accuracy: 99.007 % total_loss: 18.717
epoch: 6 correct times: 59479 training accuracy: 99.132 % total_loss: 16.130
epoch: 7 correct times: 59556 training accuracy: 99.260 % total_loss: 13.556
epoch: 8 correct times: 59636 training accuracy: 99.393 % total_loss: 11.578
epoch: 9 correct times: 59651 training accuracy: 99.418 % total_loss: 10.386


In [ ]:
# total_epochs = 10

# for epoch in range(total_epochs):  # 训练周期
#     total_loss = 0
#     total_train_correct = 0

#     for batch in train_loader:  # get a batch from the dataloader
        
#         # 读取样本数据，完成正向传播，计算损失,我们取出一个 batch 的样本，传递给 network，并计算损失。
#         images, labels = batch
#         preds = network(images)
#         loss = loss_func(preds, labels)
        
#         # 将权重的偏导重新归零；如果不归零，那么在反向传播时，计算出来的偏导会累加在原先的偏导上，造成错误。
#         optimizer.zero_grad()

#         # torch包中实现反向传播
#         loss.backward()
        
#         # torch包中实现更新参数 
#         optimizer.step()
  
#         total_loss += loss.item()
#         total_train_correct += get_num_correct(preds, labels)
    
#     print("epoch:", epoch, 
#           "correct times:", total_train_correct,
#           f"training accuracy:", "%.3f" %(total_train_correct/len(train_set)*100), "%", 
#           "total_loss:", "%.3f" %total_loss)

<font color=black size=3 face=雅黑>　　训练结果看上去不错，让我们在测试集上看看准确率如何。
    
<font color=black size=3 face=雅黑>　　**请将测试结果保存，并展示在实验报告中。**    
 

In [76]:
total_test_correct = 0
total_loss = 0

for batch in test_loader:  # get a batch from the dataloader
    images, labels = batch
    preds = network(images)
    loss = loss_func(preds, labels)

    total_loss += loss
    total_test_correct += get_num_correct(preds, labels)
    
print("correct times:", total_test_correct, 
      f"test accuracy:", "%.3f" %(total_test_correct/len(test_set)*100), "%",
      "total_loss:", "%.3f" %total_loss)

correct times: 9897 test accuracy: 98.970 % total_loss: 3.132


<br>

## 5.  知识整合及应用：自定义神经网络在 Fashion-MNIST 上的训练与测试

<br>

<font color=black size=3 face=雅黑>　　至此，我们已经完整学习了如何基于 PyTorch 构建一个简单的卷积神经网络，并在数据集上训练和测试。为了加深大家对每一步的理解，之前我们将整个过程拆分开来细讲。在这一部分中，我们会把之前学的内容串起来，突出重点。
    
<font color=black size=3 face=雅黑>　　下面，请同学们将上述知识进行整合，自行设计一个7层神经网络（含三个卷积层和三个全连接层，卷积层可以尝试将 padding 设置为非零值），并在 Fashion-MNIST 数据集上进行训练和测试。
    
<font color=black size=3 face=雅黑>　　目标：测试集上准确率高于 87\%。

<br>

<font color=black size=3 face=雅黑>1) 　读取和处理数据集

In [27]:
import torchvision
import torch
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

batch_size = 40
train_set = torchvision.datasets.FashionMNIST(
    root="./FashionMNIST", train=True, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
)
test_set = torchvision.datasets.FashionMNIST(
    root="./FashionMNIST", train=False, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size, shuffle=True
)


<font color=black size=3 face=雅黑>2) 　自定义神经网络

In [28]:
class Network3(nn.Module):
    def __init__(self):
        super(Network3, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=2, padding=1)

        self.fc1 = nn.Linear(in_features=96,  # 输入激活向量的长度
                             out_features=120)  # 输出激活向量的长度

        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # conv1
        t = self.conv1(t)
        t = F.relu(t)  # 这里我们用到了 nn.functional 包中的激活函数 relu
        t = F.max_pool2d(t, kernel_size=2, stride=2)  # 这里我们用到了 nn.functional 包中的池化函数 max_pool2d （max pooling）

        # conv2
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # conv3
        t = F.relu(self.conv3(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # 展平
        t = t.reshape((t.shape[0], -1))

        # fc1
        t = self.fc1(t)
        t = F.relu(t)

        # fc2
        t = self.fc2(t)
        t = F.relu(t)

        # output layer
        t = self.out(t)

        return t


network = Network3()

<font color=black size=3 face=雅黑>3) 　损失函数和准确率函数

In [29]:
loss_func = nn.CrossEntropyLoss()
def get_num_correct(preds, labels):  # get the number of correct times
    return preds.argmax(dim=1).eq(labels).sum().item()

<font color=black size=3 face=雅黑>4) 　优化算法

In [30]:
optimizer = optim.SGD(network.parameters(), lr=0.1)  # lr: 学习率

<font color=black size=3 face=雅黑>5) 　训练

In [ ]:
total_epochs = 100

for epoch in range(total_epochs):  # 训练周期

    total_loss = 0
    total_train_correct = 0

    for batch in train_loader:  # get a batch from the dataloader

        # 读取样本数据，完成正向传播，计算损失

        ##################### please finish the code ########################

        # images, labels = XXX
        # preds = XXX
        # loss = XXX
        images, labels = batch
        preds = network(images)
        loss = loss_func(preds, labels)
        ################################ end ################################

        # 下面这行非常重要，它使得优化器 (optimizer) 将权重的偏导重新归零；
        # 如果不归零，那么在反向传播时，计算出来的偏导会累加在原先的偏导上，
        # 造成错误。
        optimizer.zero_grad()

        # 完成反向传播，更新参数

        ##################### please finish the code ########################

        # 反向传播 （一行代码）
        loss.backward()

        # 更新参数 （一行代码）
        optimizer.step()

        ################################ end ################################

        total_loss += loss.item()
        total_train_correct += get_num_correct(preds, labels)

    print("epoch:", epoch,
          "correct times:", total_train_correct,
          f"training accuracy:", "%.3f" % (total_train_correct / len(train_set) * 100), "%",
          "total_loss:", "%.3f" % total_loss)

epoch: 0 correct times: 36090 training accuracy: 60.150 % total_loss: 1571.385
epoch: 1 correct times: 48363 training accuracy: 80.605 % total_loss: 765.990
epoch: 2 correct times: 50343 training accuracy: 83.905 % total_loss: 645.171
epoch: 3 correct times: 51275 training accuracy: 85.458 % total_loss: 581.637
epoch: 4 correct times: 51809 training accuracy: 86.348 % total_loss: 543.634
epoch: 5 correct times: 52193 training accuracy: 86.988 % total_loss: 514.942
epoch: 6 correct times: 52622 training accuracy: 87.703 % total_loss: 490.967
epoch: 7 correct times: 52754 training accuracy: 87.923 % total_loss: 477.049
epoch: 8 correct times: 52946 training accuracy: 88.243 % total_loss: 464.435
epoch: 9 correct times: 53183 training accuracy: 88.638 % total_loss: 450.689
epoch: 10 correct times: 53276 training accuracy: 88.793 % total_loss: 441.395
epoch: 11 correct times: 53410 training accuracy: 89.017 % total_loss: 431.796
epoch: 12 correct times: 53531 training accuracy: 89.218 % to

<font color=black size=3 face=雅黑>6) 　测试

In [ ]:
total_test_correct = 0
total_loss = 0

for batch in test_loader:  # get a batch from the dataloader
    images, labels = batch
    preds = network(images)
    loss = loss_func(preds, labels)

    total_loss += loss
    total_test_correct += get_num_correct(preds, labels)

print("correct times:", total_test_correct,
      f"test accuracy:", "%.3f" % (total_test_correct / len(test_set) * 100), "%",
      "total_loss:", "%.3f" % total_loss)

<br>

# 5.  实验报告

<br>

<font color=black size=3 face=雅黑>请同学们在实验报告中完成如下内容：
    
<font color=black size=3 face=雅黑>（关于代码：请大家将对应部分的代码和实验结果截图贴在实验报告中并进行相关说明即可，不要求上传完整的ipynb文件。如果大家在实验中进行了其他探索和尝试，也欢迎将其加在实验报告中。）

<br>
    
<font color=black size=3 face=雅黑>- “0_pytorch_basics.ipynb”：5. Tensor 综合练习。（请完成代码，展示结果，并对代码和结果进行说明，回答相关问题。）
    
<font color=black size=3 face=雅黑>- “1_mnist_dataset_import.ipynb”：2. 读取并处理数据集 MNIST。（请完成对 train_set 相关代码的解释说明，并补全 test_set 的代码。）
    
<font color=black size=3 face=雅黑>- “2_python_cnn.ipynb”：3. 定义一个神经网络。（相关内容和要求已在文档中加粗。）
    
<font color=black size=3 face=雅黑>- “2_python_cnn.ipynb”：4. 训练神经网络。（相关内容和要求已在文档中加粗。）
    
    
<font color=black size=3 face=雅黑>- “2_python_cnn.ipynb”：5. 知识整合及应用。（请完成代码，展示结果，并做出说明。）

<br>

In [ ]:
total_epochs = 20

for epoch in range(total_epochs):  # 训练周期

    total_loss = 0
    total_train_correct = 0

    for batch in train_loader:  # get a batch from the dataloader

        # 读取样本数据，完成正向传播，计算损失
        images, labels = batch
        preds = network(images)
        loss = loss_func(preds, labels)

        # 将权重的偏导重新归零；如果不归零，那么在反向传播时，计算出来的偏导会累加在原先的偏导上，造成错误。
        optimizer.zero_grad()
        # 反向传播 
        loss.backward()
        # 更新参数 
        optimizer.step()

        total_loss += loss.item()
        total_train_correct += get_num_correct(preds, labels)

    print("epoch:", epoch,
          "correct times:", total_train_correct,
          f"training accuracy:", "%.3f" % (total_train_correct / len(train_set) * 100), "%",
          "total_loss:", "%.3f" % total_loss)